<a href="https://colab.research.google.com/github/AlexFly666/LLM-in-Practice/blob/main/chapter09/0_quikstart/langchain_agent_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# LangChain V0.3版本
!pip install langchain-community \
        langgraph \
        langchain-openai \
        tavily-python \
        langgraph-checkpoint-sqlite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: lang

In [ ]:
# 将使用Tavily（搜索引擎）作为工具。要使用它，你需要获取并设置API密钥：
# Tavily's Search API地址 https://tavily.com/
import os
os.environ["TAVILY_API_KEY"] = "XXX" # 将 API key 设置为环境变量


In [ ]:
# 导入相关的 LangChain 功能模块
# 从 langchain_openai 库导入 ChatOpenAI 和 OpenAIEmbeddings，用于使用 OpenAI 的聊天模型和嵌入模型
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults  # 导入 Tavily 搜索引擎工具
from langchain_core.messages import HumanMessage  # 导入 HumanMessage 类型，用于表示用户输入
from langgraph.checkpoint.memory import MemorySaver  # 导入 MemorySaver，用于内存存储对话历史
from langgraph.prebuilt import create_react_agent  # 导入 create_react_agent 函数，用于创建 React Agent

In [ ]:
# 创建 Agent 的组件

# 初始化内存存储器，用于保存对话历史 (可选，用于有状态 Agent)
memory = MemorySaver()

# Create a ChatOpenAI model
# OpenAI API调用（代理方式）
model = ChatOpenAI(
    api_key="XXX",
    base_url="https://vip.apiyi.com/v1"
)


# 初始化 Tavily 搜索引擎工具，设置最大搜索结果数为 2
search = TavilySearchResults(max_results=2)
# 将 Tavily 搜索引擎工具放入工具列表，Agent 可以使用这些工具
tools = [search]

# 使用 `create_react_agent` 函数创建 Agent 执行器
# 该函数会自动将模型和工具绑定，并使用 ReAct 框架构建 Agent
# checkpointer=memory 参数用于启用对话记忆功能
agent_executor = create_react_agent(model, tools, checkpointer=memory)


# ---  首次对话：打招呼并告知所在地 ---
# 配置对话线程 ID，用于记忆功能，相同 thread_id 共享对话历史
config = {"configurable": {"thread_id": "abc123"}}
print("--- 首次对话 ---")
# 使用 stream 方法进行流式输出，逐步显示 Agent 的响应
for chunk in agent_executor.stream(
    # 构造用户输入消息 HumanMessage -  用户用中文打招呼，并告知住在北京
    {"messages": [HumanMessage(content="你好！我是小明，我住在北京。")]}, config
):
    # 打印每个 chunk 的输出，展示 Agent 的思考过程
    print(chunk)
    print("----")


# ---  第二次对话：询问天气 ---
print("\n--- 第二次对话 ---")
for chunk in agent_executor.stream(
    # 再次使用相同的 thread_id，Agent 将记住之前的对话 -  用户用中文询问天气
    {"messages": [HumanMessage(content="我住的地方天气怎么样？")]}, config
):
    print(chunk)
    print("----")


# ---  第三次对话： 询问名字 ---
print("\n--- 第三次对话 (验证记忆) ---")
for chunk in agent_executor.stream(
    # 继续使用相同的 thread_id -  用户用中文询问名字
    {"messages": [HumanMessage(content="请问我叫什么名字？")]}, config
):
    print(chunk)
    print("----")


# ---  新的对话：更换线程 ID，验证无记忆 ---
# 使用新的 thread_id，开启新的对话
config_new_thread = {"configurable": {"thread_id": "xyz123"}}
print("\n--- 新的对话 (无记忆) ---")
for chunk in agent_executor.stream(
    # 用户在新对话线程中，用中文询问名字
    {"messages": [HumanMessage(content="我叫什么名字？")]}, config_new_thread
):
    print(chunk)
    print("----")


print("\n---  代码执行完成  ---")

--- 首次对话 ---
{'agent': {'messages': [AIMessage(content='你好，小明！很高兴认识你。北京是个充满历史和文化的地方，你最喜欢那里哪个景点呢？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 93, 'total_tokens': 143, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5f9a334b-e46c-4afc-bebd-f1db22860bb9-0', usage_metadata={'input_tokens': 93, 'output_tokens': 50, 'total_tokens': 143, 'input_token_details': {}, 'output_token_details': {}})]}}
----

--- 第二次对话 ---
{'agent': {'messages': [AIMessage(content='我无法提供实时天气信息，但你可以通过天气应用或网站查询北京的最新天气情况。一般来说，北京冬季寒冷干燥，夏季炎热潮湿。你今天打算出去吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 165, 'total_tokens': 248, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish

In [1]:
!pip install -qU "langchain-community>=0.2.11" tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00


In [2]:
# 将使用Tavily（搜索引擎）作为工具。要使用它，你需要获取并设置API密钥：
# Tavily's Search API地址 https://tavily.com/
import os
os.environ["TAVILY_API_KEY"] = "XXX" # 将 API key 设置为环境变量

In [3]:
# 展示如何实例化 Tavily 搜索工具，使用
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

In [4]:
# 查询
tool.invoke({"query": "What happened at the last wimbledon"})

[{'url': 'https://www.nbcnews.com/news/sports/andy-murray-wimbledon-tennis-singles-draw-rcna159912',
  'content': "NBC News Now LONDON — Andy Murray, one of the last decade's most successful male tennis players, has pulled out of the singles tournament at what is almost certain to be his last Wimbledon, his team confirmed Tuesday. Murray, 37, who has won the Wimbledon singles title twice and the U.S Open once, has been battling to be fit to play at the All England Club for weeks. “Unfortunately, despite working incredibly hard on his recovery since his operation just over a week ago, Andy has taken the very difficult decision not to play the singles this year,” his team said in a statement reported by Sky News. The news caps a glittering career on the ATP singles tour, which placed Murray at No. 1 in the world for 41 weeks."},
 {'url': 'https://www.nytimes.com/athletic/live-blogs/wimbledon-2024-live-updates-alcaraz-djokovic-mens-final-result/kJJdTKhOgkZo/mGpXBR2QIift/',
  'content': "C

### 实战

In [6]:
!pip install -qU "langchain[openai]>=0.2.11" tavily-python langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 1.9 MB/s eta 0:00:00


In [20]:
# 将使用Tavily（搜索引擎）作为工具。要使用它，你需要获取并设置API密钥：
# Tavily's Search API地址 https://tavily.com/
import os
os.environ["TAVILY_API_KEY"] = "XXX" # 将 API key 设置为环境变量

In [24]:
# -*- coding: utf-8 -*-
"""
LangChain智能体系统实现 - 温网赛事分析

执行流程：
1. 安装依赖
2. 配置API密钥
3. 初始化搜索工具
4. 构建智能体系统
5. 执行查询分析
"""

# === 依赖安装提示 ===
"""
# 执行前需先安装依赖（取消注释运行）
# pip install -qU "langchain[openai]>=0.2.11" tavily-python langchain
"""

# === 依赖导入 ===
from datetime import datetime
import getpass
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain_community.tools import TavilySearchResults

# === 模块定义 ===
def configure_environment():
    """配置API密钥环境变量"""
    if not os.environ.get("TAVILY_API_KEY"):
        os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

    # 如需使用OpenAI官方接口，取消以下注释
    # if not os.environ.get("OPENAI_API_KEY"):
    #     os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key:\n")

def initialize_tools():
    """初始化搜索工具"""
    return TavilySearchResults(
        max_results=3,
        search_depth="advanced",
        include_answer=True,
        include_images=True
    )

def build_llm():
    """构建大语言模型实例"""
    return ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0
    )

def create_agent(tools, llm):
    """构建智能体系统"""
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "你是一个专业体育赛事分析师，当前日期：{date}"),
        ("user", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ])

    return initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.OPENAI_FUNCTIONS,
        verbose=True,
        prompt=prompt_template
    )

def execute_query(agent_executor, query):
    """执行分析查询"""
    return agent_executor.invoke({
        "input": query,
        "date": datetime.now().strftime("%Y-%m-%d")
    })

# === 主程序 ===
def main():
    # 环境配置
    configure_environment()

    # 组件初始化
    search_tool = initialize_tools()
    llm_instance = build_llm()
    agent = create_agent([search_tool], llm_instance)

    # 执行查询
    query = "分析温网女单最新赛事结果及其对选手排名的影响"
    result = execute_query(agent, query)

    # 输出结果
    print("\n=== 分析报告 ===")
    print(result["output"])

if __name__ == "__main__":
    main()



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '温网女单最新赛事结果 2023'}`


[{'url': 'https://sports.cctv.com/2023/07/15/VIDEku8DhC6oKpAzxG2RtlUU230715.shtml', 'content': '来源：央视网 更新时间：2023年07月15日 22:54 视频简介 央视网消息：北京时间7月15日，温网女单决赛展开争夺，万卓索娃2-0击败贾巴尔，夺得温网女单冠军。'}, {'url': 'https://sports.cctv.com/2023/07/16/ARTIeKtyW40OwwiUBU38yrGI230716.shtml', 'content': '央视网消息：北京时间7月15日，2023年温网女单决赛，上届亚军、6号种子贾巴尔对阵非种子选手万卓索娃。 万卓索娃直落两盘总分2-0横扫贾巴尔，勇夺温网女单冠军，也是生涯首次夺得大满贯女单冠军，成功缔造一系列纪录创造历史。'}, {'url': 'https://sports.sina.com.cn/tennis/wta/2023-07-16/doc-imzavrzx3251356.shtml', 'content': '北京时间7月16日凌晨，2023温网公开赛结束了第13比赛日的争夺。在女单决赛中，捷克非种子选手万卓索娃一黑到底，以2个6-4横扫6号种子、去年赛会'}]在2023年温布尔登网球锦标赛（温网）女单决赛中，捷克选手万卓索娃以2-0战胜了6号种子贾巴尔，成功夺得冠军。这是万卓索娃职业生涯中的首个大满贯女单冠军，同时她的胜利也创造了一系列历史纪录。

### 赛事结果分析
- **决赛结果**：万卓索娃以6-4, 6-4的比分击败贾巴尔。
- **选手背景**：万卓索娃在本次比赛中并未被列为种子选手，而贾巴尔是上届亚军和6号种子。

### 对选手排名的影响
万卓索娃的胜利将显著提升她在WTA（女子网球协会）的排名。根据WTA的积分系统，夺得大满贯赛事的冠军将为选手带来大量积分，这将直接影响她的世界排名。

- **万卓索娃的排名提升**：她的冠军积分将使她的排名大幅上升，可